# Joint Task Post-Processing

## Libraries

In [1]:
import os
import pickle
import json
import pandas as pd
import ast

In [2]:
nb_epochs = 10
output_dir = "models/PE_ATC-LI-LTC_paragraph_wo_tags_llama-3-8b-Instruct"

In [3]:
with open(os.path.join(output_dir, f"""PE_ATC-LI-LTC_results_{nb_epochs}.pickle"""), "rb") as fh:
        
        results = pickle.load(fh)

In [4]:
results

{'ground_truths': ['{"list_component_types": [], "list_argument_relations_and_types": []}',
  '{"list_component_types": ["MajorClaim"], "list_argument_relations_and_types": []}',
  '{"list_component_types": ["Premise", "Premise", "Premise", "Claim"], "list_argument_relations_and_types": [[3, 0, "Support"], [3, 1, "Support"], [3, 2, "Support"]]}',
  '{"list_component_types": ["Premise", "Premise", "Premise", "Claim"], "list_argument_relations_and_types": [[3, 0, "Support"], [3, 1, "Support"], [3, 2, "Support"]]}',
  '{"list_component_types": ["Premise", "Premise", "Claim"], "list_argument_relations_and_types": [[2, 0, "Support"], [2, 1, "Support"]]}',
  '{"list_component_types": [], "list_argument_relations_and_types": []}',
  '{"list_component_types": [], "list_argument_relations_and_types": []}',
  '{"list_component_types": ["MajorClaim"], "list_argument_relations_and_types": []}',
  '{"list_component_types": ["Premise", "Premise", "Premise", "Claim"], "list_argument_relations_and_typ

In [5]:
grounds_global = results["ground_truths"]

In [6]:
preds_global = results["predictions"]

In [7]:
len(grounds_global), len(preds_global)

(439, 439)

## ATC post-processing

In [8]:
grounds = [json.loads(x)["list_component_types"] for x in grounds_global]

In [9]:
preds = [x["content"] for x in preds_global]

In [10]:
preds = [json.loads(x)["list_component_types"] for x in preds]

In [11]:
len(grounds), len(preds)

(439, 439)

In [12]:
grounds

[[],
 ['MajorClaim'],
 ['Premise', 'Premise', 'Premise', 'Claim'],
 ['Premise', 'Premise', 'Premise', 'Claim'],
 ['Premise', 'Premise', 'Claim'],
 [],
 [],
 ['MajorClaim'],
 ['Premise', 'Premise', 'Premise', 'Claim'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Premise', 'Claim'],
 ['MajorClaim', 'Claim'],
 [],
 ['MajorClaim', 'MajorClaim'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Claim', 'Premise'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Claim', 'Premise', 'Premise'],
 ['MajorClaim'],
 [],
 ['MajorClaim'],
 ['Premise', 'Premise', 'Claim', 'Premise'],
 ['Premise', 'Premise', 'Claim'],
 ['MajorClaim', 'Claim'],
 [],
 ['MajorClaim'],
 ['Premise', 'Claim', 'Premise', 'Premise', 'Claim', 'Premise', 'Premise'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Premise', 'Premise', 'Claim'],
 ['Claim', 'Premise', 'Premise', 'Premise', 'Premise'],
 ['MajorClaim', 'Claim'],
 [],
 ['MajorClaim'],
 ['Premise', 'Premise', 'Claim'],
 ['Claim', 'Premise', 'Premise', 'Premise'],
 ['Pr

In [13]:
preds

[[],
 ['MajorClaim'],
 ['Premise', 'Premise', 'Premise', 'Claim'],
 ['Claim', 'Premise', 'Premise', 'Premise'],
 ['Claim', 'Premise', 'Premise'],
 [],
 [],
 ['MajorClaim'],
 ['Premise', 'Claim', 'Premise', 'Premise'],
 ['Claim', 'Premise', 'Premise', 'Premise', 'Premise', 'Premise'],
 ['MajorClaim', 'Claim'],
 [],
 ['MajorClaim', 'Claim'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Claim', 'Premise'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Claim', 'Premise', 'Premise'],
 ['MajorClaim'],
 [],
 ['MajorClaim'],
 ['Premise', 'Premise', 'Premise', 'Claim'],
 ['Premise', 'Premise', 'Claim'],
 ['MajorClaim', 'Claim'],
 [],
 ['MajorClaim'],
 ['Claim', 'Premise', 'Premise', 'Premise', 'Premise', 'Premise', 'Premise'],
 ['Premise', 'Premise', 'Claim', 'Premise', 'Premise', 'Premise', 'Claim'],
 ['Claim', 'Premise', 'Premise', 'Premise', 'Premise'],
 ['MajorClaim', 'Claim'],
 [],
 ['MajorClaim'],
 ['Premise', 'Premise', 'Claim'],
 ['Premise', 'Premise', 'Claim', 'Premise'],
 ['Premise

In [14]:
def opposite(component_type):

    if component_type == "Premise":
        return "Claim"
    elif component_type == "Claim":
        return "Premise"
    elif component_type == "MajorClaim":
        return "Claim"

In [15]:
def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds 

In [16]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    
    if len(x) != len(y):
        
        preds[i] = harmonize_preds(x, y)

In [17]:
ATC_preds = [item for row in preds for item in row]
ATC_grounds = [item for row in grounds for item in row]

In [18]:
# sanity check: 
len(ATC_preds) == len(ATC_grounds)

True

In [19]:
from sklearn.metrics import classification_report

In [20]:
print(classification_report(ATC_grounds, ATC_preds, digits=3))

              precision    recall  f1-score   support

       Claim      0.774     0.784     0.779       283
  MajorClaim      0.979     0.922     0.950       154
     Premise      0.916     0.923     0.919       724

    accuracy                          0.889      1161
   macro avg      0.890     0.876     0.883      1161
weighted avg      0.890     0.889     0.889      1161



In [21]:
### ARI post-processing

In [22]:
len(grounds_global), len(preds_global)

(439, 439)

In [23]:
grounds = [json.loads(x)["list_argument_relations_and_types"] for x in grounds_global]

preds = [x["content"] for x in preds_global]

preds = [json.loads(x)["list_argument_relations_and_types"] for x in preds]

In [24]:
grounds

[[],
 [],
 [[3, 0, 'Support'], [3, 1, 'Support'], [3, 2, 'Support']],
 [[3, 0, 'Support'], [3, 1, 'Support'], [3, 2, 'Support']],
 [[2, 0, 'Support'], [2, 1, 'Support']],
 [],
 [],
 [],
 [[3, 0, 'Support'], [3, 1, 'Support'], [3, 2, 'Support']],
 [[5, 0, 'Support'],
  [5, 1, 'Support'],
  [5, 2, 'Attack'],
  [2, 3, 'Attack'],
  [5, 4, 'Support']],
 [],
 [],
 [],
 [[4, 0, 'Support'],
  [0, 1, 'Support'],
  [4, 2, 'Support'],
  [4, 3, 'Support'],
  [4, 5, 'Support']],
 [[4, 0, 'Support'],
  [4, 1, 'Support'],
  [4, 2, 'Support'],
  [4, 3, 'Support'],
  [4, 5, 'Support'],
  [4, 6, 'Support']],
 [],
 [],
 [],
 [[2, 0, 'Support'], [0, 1, 'Support'], [2, 3, 'Support']],
 [[2, 0, 'Support'], [2, 1, 'Support']],
 [],
 [],
 [],
 [[1, 0, 'Support'],
  [1, 2, 'Support'],
  [4, 3, 'Attack'],
  [4, 5, 'Support'],
  [4, 6, 'Support']],
 [[2, 0, 'Support'],
  [2, 1, 'Support'],
  [6, 2, 'Support'],
  [6, 3, 'Support'],
  [3, 4, 'Support'],
  [3, 5, 'Support']],
 [[0, 1, 'Support'], [0, 2, 'Support'],

In [25]:
preds

[[],
 [],
 [[3, 0, 'Support'], [3, 1, 'Support'], [3, 2, 'Support']],
 [[0, 1, 'Support'], [3, 2, 'Support'], [0, 3, 'Support']],
 [[0, 1, 'Support'], [0, 2, 'Support']],
 [],
 [],
 [],
 [[1, 0, 'Support'], [1, 2, 'Support'], [1, 3, 'Support']],
 [[0, 1, 'Support'],
  [0, 2, 'Attack'],
  [2, 3, 'Attack'],
  [0, 4, 'Support'],
  [0, 5, 'Support']],
 [],
 [],
 [],
 [[4, 0, 'Support'],
  [4, 1, 'Support'],
  [4, 2, 'Support'],
  [4, 3, 'Support'],
  [4, 5, 'Support']],
 [[4, 0, 'Support'],
  [4, 1, 'Support'],
  [4, 2, 'Support'],
  [4, 3, 'Support'],
  [4, 5, 'Support'],
  [4, 6, 'Support']],
 [],
 [],
 [],
 [[3, 0, 'Support'], [2, 1, 'Support'], [3, 2, 'Support']],
 [[2, 0, 'Support'], [2, 1, 'Support']],
 [],
 [],
 [],
 [[0, 1, 'Support'],
  [1, 2, 'Support'],
  [0, 3, 'Attack'],
  [3, 4, 'Attack'],
  [0, 5, 'Support'],
  [0, 6, 'Support']],
 [[2, 0, 'Support'],
  [2, 1, 'Support'],
  [6, 3, 'Support'],
  [3, 4, 'Support'],
  [3, 5, 'Support']],
 [[0, 1, 'Support'], [0, 2, 'Support'], 

In [26]:
def get_ar_pairs(grounds):

    grounds_l = []
    
    for sublist in grounds:
        l_tmp = []
        for l in sublist:
            l_tmp.append(l[0:2])
            
        grounds_l.append(l_tmp)

    return grounds_l

In [27]:
grounds = get_ar_pairs(grounds)
preds = get_ar_pairs(preds)

In [28]:
len(grounds), len(preds)

(439, 439)

In [29]:
pe_df = pd.read_csv(os.path.abspath("datasets/PE_data.csv"))

# pe_df

df_split = pd.read_csv("datasets/train-test-split.csv", sep=";")

pe_df['split'] = pe_df['essay_id'].map(df_split['SET'])

In [30]:
def get_ac_count(x):

    return len(ast.literal_eval(x.AC_types))

In [31]:
pe_df["AC_count"] = pe_df.apply(lambda x: get_ac_count(x), axis=1)

In [32]:
def get_all_possible_pairs(x):

    pairs_l = []

    ac_count = x.AC_count
    ar_pairs = ast.literal_eval(x.AR_pairs)
    
    for i in range(ac_count):
        for j in range(ac_count):
            if i != j:
                pair = (i, j)
                if pair in ar_pairs:
                    pairs_l.append([i, j, "Rel"])
                else:
                    pairs_l.append([i, j, "N-Rel"])

    return pairs_l

In [33]:
pe_df["LI_grounds"] = pe_df.apply(lambda x: get_all_possible_pairs(x), axis=1)

In [34]:
LI_grounds_l = list(pe_df[pe_df.split == "TEST"].reset_index().LI_grounds)

In [35]:
def process_preds(test_preds, LI_grounds_l):

    if len(test_preds) != len(LI_grounds_l):
        print("error in preds length")

    preds_pairs_l = []

    for l_1, l_2 in zip(test_preds, LI_grounds_l):
        para_list = []
        for i, j, _ in l_2:
            pair = [i, j]
            if pair in l_1:
                para_list.append([i, j, "Rel"])
            else:
                para_list.append([i, j, "N-Rel"])

        preds_pairs_l.append(para_list)

    return preds_pairs_l

In [36]:
LI_preds_l = process_preds(preds, LI_grounds_l)

In [37]:
len(LI_grounds_l) == len(LI_preds_l)

True

In [38]:
LI_preds = [item for row in LI_preds_l for item in row]
LI_grounds = [item for row in LI_grounds_l for item in row]

In [39]:
# sanity check: 
len(LI_preds) == len(LI_grounds)

True

In [40]:
LI_preds = [x[2] for x in LI_preds]
LI_grounds = [x[2] for x in LI_grounds]

In [41]:
print(classification_report(LI_grounds, LI_preds, digits=3))

              precision    recall  f1-score   support

       N-Rel      0.933     0.932     0.933      3384
         Rel      0.684     0.688     0.686       724

    accuracy                          0.889      4108
   macro avg      0.809     0.810     0.809      4108
weighted avg      0.889     0.889     0.889      4108



## ARC post-processing

In [83]:
grounds = [json.loads(x)["list_argument_relations_and_types"] for x in grounds_global]

preds = [x["content"] for x in preds_global]

preds = [json.loads(x)["list_argument_relations_and_types"] for x in preds]

In [84]:
preds

[[],
 [],
 [[3, 0, 'Support'], [3, 1, 'Support'], [3, 2, 'Support']],
 [[0, 1, 'Support'], [3, 2, 'Support'], [0, 3, 'Support']],
 [[0, 1, 'Support'], [0, 2, 'Support']],
 [],
 [],
 [],
 [[1, 0, 'Support'], [1, 2, 'Support'], [1, 3, 'Support']],
 [[0, 1, 'Support'],
  [0, 2, 'Attack'],
  [2, 3, 'Attack'],
  [0, 4, 'Support'],
  [0, 5, 'Support']],
 [],
 [],
 [],
 [[4, 0, 'Support'],
  [4, 1, 'Support'],
  [4, 2, 'Support'],
  [4, 3, 'Support'],
  [4, 5, 'Support']],
 [[4, 0, 'Support'],
  [4, 1, 'Support'],
  [4, 2, 'Support'],
  [4, 3, 'Support'],
  [4, 5, 'Support'],
  [4, 6, 'Support']],
 [],
 [],
 [],
 [[3, 0, 'Support'], [2, 1, 'Support'], [3, 2, 'Support']],
 [[2, 0, 'Support'], [2, 1, 'Support']],
 [],
 [],
 [],
 [[0, 1, 'Support'],
  [1, 2, 'Support'],
  [0, 3, 'Attack'],
  [3, 4, 'Attack'],
  [0, 5, 'Support'],
  [0, 6, 'Support']],
 [[2, 0, 'Support'],
  [2, 1, 'Support'],
  [6, 3, 'Support'],
  [3, 4, 'Support'],
  [3, 5, 'Support']],
 [[0, 1, 'Support'], [0, 2, 'Support'], 

In [85]:
def get_ar_pairs(grounds):

    grounds_l = []
    
    for sublist in grounds:
        l_tmp = []
        for l in sublist:
            try:
                l_tmp.append(l[2])
            except:
                l_tmp.append("Support")
            
        grounds_l.append(l_tmp)

    return grounds_l

In [86]:
grounds = get_ar_pairs(grounds)
preds = get_ar_pairs(preds)

In [87]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    if len(x) != len(y):
        print(i,x,y)

23 ['Support', 'Support', 'Attack', 'Support', 'Support'] ['Support', 'Support', 'Attack', 'Attack', 'Support', 'Support']
24 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'] ['Support', 'Support', 'Support', 'Support', 'Support']
60 [] ['Attack']
72 [] ['Support']
75 ['Support'] ['Attack', 'Attack']
91 [] ['Support']
106 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'] ['Support', 'Support', 'Support', 'Support', 'Support', 'Support', 'Support']
107 ['Support', 'Support', 'Support', 'Support'] ['Support', 'Support', 'Support']
120 [] ['Attack']
160 ['Support', 'Support'] ['Support', 'Support', 'Support']
169 ['Support', 'Support'] ['Support']
223 ['Support', 'Support', 'Support'] []
228 ['Support', 'Support'] ['Support', 'Support', 'Support']
241 ['Support', 'Support', 'Support'] ['Support', 'Support']
247 ['Support', 'Support', 'Support', 'Support', 'Support', 'Support'] ['Support', 'Support', 'Support', 'Support', 'Support']
259 ['Support', 'Suppo

In [88]:
def opposite(relation_type):
    
    if relation_type == "Support":
        return "Attack"
    else:
        return "Support"

In [89]:
def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds  

In [90]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    
    if len(x) != len(y):
        
        preds[i] = harmonize_preds(x, y)

In [91]:
LTC_preds = [item for row in preds for item in row]
LTC_grounds = [item for row in grounds for item in row]

In [92]:
# sanity check: 
len(LTC_preds) == len(LTC_grounds)

True

In [93]:
print(classification_report(LTC_grounds, LTC_preds, digits=3))

              precision    recall  f1-score   support

      Attack      0.617     0.630     0.624        46
     Support      0.975     0.973     0.974       678

    accuracy                          0.952       724
   macro avg      0.796     0.802     0.799       724
weighted avg      0.952     0.952     0.952       724

